In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

rng = np.random.default_rng()

# Setting Initial Parametres. 

L_size = 20
set_site_matrix = np.array([0] * (L_size*L_size)).reshape((L_size, L_size))
q_spin = [-1, 1] 
k_B = 1  
J = 1
T_bar = 0.1
n_0 = 100
T_arr = [0.1, 0.5, 1, 1.5, 2, 2.25, 2.5, 3, 10]


def randomize_site(c, L_size):

    for i in range(L_size):
        for j in range(L_size):
            set_site_matrix[i, j] = q_spin[rng.integers(2)]
    return set_site_matrix
        
def energy_calculation(L_size, set_site_matrix):

    energy_sum = 0

    for i in range(L_size):
        for j in range(L_size):
            energy_sum += set_site_matrix[i][j] * (set_site_matrix[i][(j+1)%L_size] + set_site_matrix[(i+1)%L_size][j])
    
    return -1 * energy_sum

def magnetization_calulation(L_size, set_site_matrix):

    magnetization_per_site = 1/(L_size * L_size) * np.abs(np.sum(set_site_matrix))
    
    return magnetization_per_site


def site_update(L_size, set_site_matrix, T_bar):

    i = np.random.randint(0, L_size)
    
    j = np.random.randint(0, L_size)

    site_point = set_site_matrix[i][j]

    energy_change = 2 * site_point  * (set_site_matrix[i][(j+1) % L_size] + set_site_matrix[(i+1) % L_size][j] + set_site_matrix[(i-1) % L_size][j] + set_site_matrix[i][(j-1) % L_size])
        
    r = np.exp(-energy_change / T_bar)
    
    if r > 1 or r == 1:
        set_site_matrix[i][j] *= -1
    
    elif np.random.uniform() < r:
        set_site_matrix[i][j] *= -1
    
    return set_site_matrix


def system_update(L_size, set_site_matrix, T_bar):

    for i in range(L_size**2):

        set_site_matrix = site_update(L_size, set_site_matrix, T_bar)
    
    return set_site_matrix


def plot_configuration(L_size, set_site_matrix):
  
    for i in range(L_size):
        for j in range(L_size):

            if set_site_matrix[i,j] == 1:

                print(" 1",end="")
            
            else:
                
                print("-1 ",end="")
        
        print("")


def plot_for_data1(x_axis, energy_values_arr, magnetization_value_arr):

    fig, ax1 = plt.subplots()
    fig.suptitle('Energy and Magnetization for T = %s K, L_size = %s' %(T_bar, L_size))
    ax1.set_xlabel('Time')
    ax1.set_ylabel("E", color='red')
    ax1.plot(x_axis, energy_values_arr, color='red', label='Energy Value')
    ax1.tick_params(axis='y', labelcolor='red')
    plt.legend()
    ax2 = ax1.twinx()  
    ax2.set_ylabel('magnetization_arr_avg', color='blue') 
    ax2.plot(x_axis, magnetization_value_arr, color='blue', label='Magnetization Value')
    ax2.tick_params(axis = 'y', labelcolor='blue')
    plt.legend()
    plt.show()


def plot_for_data2(T_arr, energy_values_arr, magnetization_value_arr):

    fig, ax1 = plt.subplots()
    fig.suptitle('Energy and magnetization with respect to Temperature')
    color = 'tab:red'
    ax1.set_xlabel('Temperature')
    ax1.set_ylabel("$E'$", color=color)
    ax1.plot(T_arr, energy_values_arr, color=color, label='Energy')
    ax1.tick_params(axis='y', labelcolor=color)
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('$|magnetization_arr_avg|$', color=color) 
    ax2.plot(T_arr, magnetization_value_arr, color=color, label='magnetization')
    ax2.tick_params(axis='y', labelcolor=color)
    plt.show()

def plot_for_data3(T_arr,error_comparision_arr):

    plt.plot(T_arr, error_comparision_arr)
    plt.xlabel('Temperature')
    plt.ylabel('Error')
    plt.title("Error comparision in Metropolis Algorithm and to Onsager's solution.")
    plt.show()


def plot_for_data4(T_arr, energy_arr_avg, energy_min_arr, energy_max_arr, magnetization_arr_avg, magnetization_max_arr, magnetization_min_arr):

    fig, ax1 = plt.subplots()
    fig.suptitle('Energy and magnetization error bars.')

    ax1.set_xlabel('Temperature')
    ax1.set_ylabel("E'", color = 'red')
    ax1.tick_params(axis = 'y', labelcolor = 'red')

    ax2 = ax1.twinx()  
    ax2.set_ylabel('magnetization_arr_avg', color='blue') 
    ax2.tick_params(axis='y', labelcolor = 'blue')

    ax1.errorbar(T_arr[:-1], energy_arr_avg[:-1], yerr=[energy_min_arr, energy_max_arr], fmt='ro', capsize=1, ecolor='red')
    ax2.errorbar(T_arr[:-1], magnetization_arr_avg, yerr=[magnetization_max_arr, magnetization_min_arr], fmt='bo', capsize=1, ecolor='black')
    
    plt.show()



def reaching_convergence(L_size, set_site_matrix, T_bar):

    energy_values_arr = []
    magnetization_value_arr = []
    
    for i in range(n_0):

        set_site_matrix = system_update(L_size, set_site_matrix, T_bar)

        energy_values_arr = np.append(energy_values_arr, energy_calculation(L_size, set_site_matrix))
        magnetization_value_arr = np.append(magnetization_value_arr, magnetization_calulation(L_size, set_site_matrix))

    n_min = {0.1:50, 0.5:50, 1:50, 1.5:40, 2:30, 2.25:30, 2.5:50, 3: 30, 10:20}

    avg_energy = (np.sum(energy_values_arr[n_min[T_bar]:])) * (1/(n_0 - n_min[T_bar]))
    avg_magnetization = (np.sum(magnetization_value_arr[n_min[T_bar]:])) * (1/(n_0 - n_min[T_bar]))

    x_axis = np.arange(0, n_0, 1)

    plot_for_data1(x_axis, energy_values_arr, magnetization_value_arr)

    return set_site_matrix, avg_energy, avg_magnetization




def ising_model(L_size, set_site_matrix, n_0, T_arr):
    
    energy_values_arr = []
    magnetization_value_arr = []

    for i in T_arr:

        set_site_matrix, E, magnetization_arr_avg = reaching_convergence(L_size, set_site_matrix, i)
        
        plot_configuration(L_size, set_site_matrix)
        print("Energy:", E)
        print("Magnetization:", magnetization_arr_avg)
        
        energy_values_arr = np.append(energy_values_arr, E)
        magnetization_value_arr = np.append(magnetization_value_arr, magnetization_arr_avg)
        set_site_matrix = randomize_site(set_site_matrix, L_size)

    plot_for_data2(T_arr, energy_values_arr, magnetization_value_arr)

    return energy_values_arr, magnetization_value_arr





def analytical_comparison(L_size, T_arr, magnetization_value_arr):

    T_c = 2 / (np.log(1 + np.sqrt(2)))

    error_comparision_arr = []

    l1 = len(T_arr)

    for i in range(l1):

        if T_arr[i] < T_c:
            m_analytical = math.pow((1-math.pow(np.sinh(2/T_arr[i]), -4)), 1/8)
  
        else:
            m_analytical = 0
        
        error_comparision_arr = np.append(error_comparision_arr , np.abs(m_analytical - magnetization_value_arr[i]))
    
    plot_for_data3(T_arr,error_comparision_arr)

    return error_comparision_arr




def error_bars_quantitative(L_size):

    seeds_arr = [5, 7, 11, 13, 17]

    energy_values_arr = []
    magnetization_value_arr = []
    
    energy_arr_avg = []
    energy_min_arr = []
    energy_max_arr = []
    magnetization_arr_avg = [] 
    magnetization_min_arr = []
    magnetization_max_arr = []

    l1 = len(T_arr)
    l2 = len(seeds_arr)

    for i in range(l2):
      
        set_site_matrix = np.zeros((L_size, L_size))
        set_site_matrix = randomize_site(set_site_matrix, L_size)
        seed_arr = seeds_arr[i]
        
        energy_values_arr, magnetization_value_arr = ising_model(L_size, set_site_matrix, n_0, T_arr)

    
    energy_values_arr, magnetization_value_arr = np.array(energy_values_arr), np.array(magnetization_value_arr)
    
    for i in range(l1):
        try:
  
          energy_arr_avg = np.append(energy_arr_avg, np.mean(energy_values_arr[:i]))

          energy_min_arr = np.append(energy_min_arr, np.min(energy_values_arr[:i]))
          energy_max_arr = np.append(energy_max_arr, np.max(energy_values_arr[:i]))

          magnetization_arr_avg = np.append(magnetization_arr_avg, np.mean(magnetization_value_arr[:i]))
          magnetization_min_arr = np.append(magnetization_min_arr, np.min(magnetization_value_arr[:i]))
          magnetization_max_arr = np.append(magnetization_max_arr, np.max(magnetization_value_arr[:i]))

        except ValueError: 
          pass

    plot_for_data4(T_arr, energy_arr_avg, energy_min_arr, energy_max_arr, magnetization_arr_avg, magnetization_max_arr, magnetization_min_arr)


def large_L_arr():
    
    L_size = [5, 10, 20, 40]
    l4 = len(L_size)

    for i in range(l4):
      error_bars_quantitative(L_size[i])

# Calling all the functions.

# System update

set_site_matrix = randomize_site(set_site_matrix, L_size)
system_update(L_size, set_site_matrix, T_bar)
print("System updated")


# Calling function for T = 0.1 and n_0 = 100 and comparing it, Ising Model and Analytical Comparison.

set_site_matrix, E, magnetization_arr_avg = reaching_convergence(L_size, set_site_matrix, T_bar)
set_site_matrix = randomize_site(set_site_matrix, L_size)
print("Energy:", E)
print("Magnetization:", magnetization_arr_avg)

set_site_matrix = randomize_site(set_site_matrix, L_size)
energy_values_arr, magnetization_value_arr = ising_model(L_size, set_site_matrix, n_0, T_arr)

# Comparing the solution.

analytical_comparison(L_size, T_arr, magnetization_value_arr)


# Setting Error bars for different L size.

large_L_arr()



Output hidden; open in https://colab.research.google.com to view.

Report 

Sub-Project 4

#### Ising model using Metropolis Algorithm.


A square lattice was formed of size L x L, the sites were set with points were set with either spin up or spin down. 

Starting by setting T = 0 , the  ground state energy was calculated and was found to be in the range of -500 J to -800 J, which can represented in the range but as the, energy decreases so does the magnetic field, from the data. From basics, if all the magnetic spin is aligned in the same direction, the substance is considered ferromagnetic. In this case either the spin can be -1 or 1 depending on the polarizing force field applied. Normally, the excited state has the energy of 4 J or - 4 J, thus the total energy being in equal to 8 J. 

As the T goes to infinity, the  neighbouring spin becomes 0 as the probability is 1. This gives us knowledge that the spin lattice is not aligned, which also sets energy and the magnetic field to zero.

The plot of the final configuration for each $T'$ is printed. 

From the graph of Energy and Magnetization with respect to temperature, it is observed that as the energy increases with increasing temperature, the magnetization decreases.

#### Finding the minimum value of n, for an array of T' : $n_{min}$

When the simulation is run using different values of the $n_o$ at different T', the values of the n' are determined by passing all the values in the array of the T'. The behaviour seems to chage for the smaller values of T' in the array but for the larger values the change is very small to the point that it is impossible to detect. 


#### Onsager's Approach Solution 

As the values of the T increased so did the error numbers, smaller values of the T showed very little change in the error.

#### Quantifying $n_{min}$ and errors depending on $L$


One thing was evidently clear, that as the value of the n_min increased the so did the size of the error bars, giving us idea that the error rages were higher, but for the largest values of the T in the array, the program was not executing as the size would overflow. Thus, results were neglected.


